In [2]:
import os

from os.path import join



from praatio import tgio





# path = join('.', 'files')

# outputPath = join(path, "merged_textgrids")

# if not os.path.exists(outputPath):

#     os.mkdir(outputPath)



# Let's use praatio to construct some hypothetical textgrids

tg = tgio.openTextgrid("bobby_words.TextGrid")

wordTier = tg.tierDict["word"]

entryList = wordTier.entryList



bobbyPhoneTG = tgio.openTextgrid("bobby_phones.TextGrid")





bobbyTG = tgio.Textgrid()

bobbyTG.addTier(bobbyPhoneTG.tierDict["phone"])

bobbyTG.addTier(tgio.IntervalTier("nouns", [entryList[1], ]))

bobbyTG.addTier(tgio.IntervalTier("verbs", [entryList[2], ]))

bobbyTG.addTier(tgio.IntervalTier("subjects", entryList[3:5]))



# Let's save it, in case you want to see it

bobbyTG.save("mergeExample_bobby_words_split.TextGrid")





# And we'll do the same for mary's textgrid

tg = tgio.openTextgrid("mary.TextGrid")

wordTier = tg.tierDict["word"]

entryList = wordTier.entryList



maryTG = tgio.Textgrid()

maryTG.addTier(tg.tierDict["phone"])

maryTG.addTier(tgio.IntervalTier("nouns", [entryList[0], ]))

maryTG.addTier(tgio.IntervalTier("verbs", [entryList[1], ]))

maryTG.addTier(tgio.IntervalTier("subjects", entryList[2:4]))



maryTG.save("mergeExample_mary_words_split.TextGrid")


# Let's combine Mary and Bob's textgrids

combinedTG = bobbyTG.appendTextgrid(maryTG, True)

combinedTG.save("mergeExample_mary_and_bob_words_split.TextGrid")



# And now let's merge their tiers together

# We'll go with the default merge function which accepts all labels,

# except silence. Any non-silent intervals that overlap will be merged

# together into a super interval

mergedTG = combinedTG.mergeTiers(tierList=["nouns", "verbs", "subjects"],

                                 preserveOtherTiers=True)



mergedTG.save("mergeExample_mary_and_bob_words_joined.TextGrid")

In [3]:
# big textgrid for the corpus

from praatio import tgio
from functools import wraps

filenames = ['game_885-1785s_UPD.TextGrid', 'game_1785-2605s_UPD.TextGrid', 'game_2605-3500s_UPD.TextGrid', 'game_3500-4400_UPD.TextGrid', 'game_4400-5300_UPD.TextGrid']
# 'game_1-885s.TextGrid', 
a = tgio.openTextgrid('game_1-885s_U.TextGrid')
for file in filenames:
    b = tgio.openTextgrid(file)
    combinedTG = a.appendTextgrid(b, True)
combinedTG.save("all_2.TextGrid")

In [34]:
# extra

from praatio import tgio
from functools import wraps
import re
import os

tg = tgio.openTextgrid('game_1-885s.TextGrid')
replicas = []
for i in range (0, 16, 5):
    b = tg.tierDict[tg.tierNameList[i]]
    replicas.extend([entry for entry in b.entryList])

a = sorted([start for start, end, label in replicas])

l = []
for item in replicas:
    for i in range (0, 16, 5):
        a = tg.tierDict[tg.tierNameList[i]]
        if item in a.entryList:
            l.extend([label for start, end, label in a.entryList])

with open('reader.txt', 'w', encoding='utf-8') as f:
    for item in l:
        f.write(item)
        f.write('\n')
        

    


In [1]:
# extra
import sqlite3

from praatio import tgio
from functools import wraps
import re
import os

# + language
def replics_db():
    conn = sqlite3.connect('replics17.db')
    c = conn.cursor()
    c.execute('CREATE TABLE IF NOT EXISTS replics(start_point float, replic text, speaker integer, inter integer)')
    cster_list = []
    cstra_list = []
    tg = tgio.openTextgrid('game_1-885s.TextGrid')
    cs_inter = tg.tierDict[tg.tierNameList[20]]
    cster_list.extend([entry[0] for entry in cs_inter.entryList])
    cs_intra = tg.tierDict[tg.tierNameList[21]]
    cstra_list.extend([entry[0] for entry in cs_intra.entryList])
    print(cstra_list)
    for i in range (0, 16, 5):
        starting_point = []
        replics = []
        a = tg.tierDict[tg.tierNameList[i]]
        b = tg.tierDict[tg.tierNameList[i + 2]]
        for s_point, e_point, label in a.entryList:
            ter = 0
            if s_point in cster_list:
                ter = 1
            c.execute('INSERT INTO replics VALUES (?, ?, ?, ?)', (s_point, label, i, ter))     
            
        conn.commit()

    conn.close()
    
replics_db()    

# SELECT * FROM replics ORDER BY start_point ASC

[1.0047227485980545, 46.57768707482993, 86.29227019139503, 94.5334586738905, 95.9918820861678, 136.62956534676968, 172.39864328428126, 172.8288688669446, 211.92182053519895, 235.74269841269842, 259.6243764172336, 261.674521177852, 286.4873242630386, 315.76292517006806, 332.58764172335606, 342.78144784797786, 369.59572806883546, 369.991988481624, 371.5564806970928, 371.86281179138325, 375.6831746031746, 390.7377590792541, 407.9806575963719, 416.96369614512474, 522.6826429753611, 525.4514739229026, 547.026507936508, 565.7937117506963, 566.1729895743653, 578.5072376591202, 621.5293197278912, 623.5171882086169, 623.7269299915873, 646.7176759778106, 650.1960850988468, 670.9085487528345, 728.5191836734695, 729.1046485260771, 731.5946768772515, 732.0644671201815, 756.8143558792862, 757.7186167800454, 809.8607896912098, 810.2762396545618, 862.4432426303855, 862.9656712793623, 863.3634920634921, 875.2132123122656, 875.6899546485262]


In [1]:
# new database for ordered db
import sqlite3

conn = sqlite3.connect('new8.db')
c = conn.cursor()
c.execute('CREATE TABLE IF NOT EXISTS new(start_point float, replic text, speaker integer, language text, inter integer)')
conn.close()

In [2]:
# database for starting_point (for turns) - replic - speaker - language - inter CS
# already ordered
import sqlite3

from praatio import tgio
from functools import wraps
import re
import os

# + language
def replics_db():
    conn = sqlite3.connect('replics22.db')
    c = conn.cursor()
    c.execute('CREATE TABLE IF NOT EXISTS replics(start_point float, replic text, speaker integer, language text, inter integer)')
    cster_list = []
#     cstra_list = []
    tg = tgio.openTextgrid('all_1.TextGrid')
    cs_inter = tg.tierDict[tg.tierNameList[20]]
    cster_list.extend([entry[0] for entry in cs_inter.entryList])
    for i in range (0, 16, 5):
        starting_point = []
        replics = []
        a = tg.tierDict[tg.tierNameList[i]]
        rus = tg.tierDict[tg.tierNameList[i + 1]]
        ykt = tg.tierDict[tg.tierNameList[i + 2]]
        rus_full_s = [entry[0] for entry in rus.entryList]
        rus_full_e = [entry[1] for entry in rus.entryList]
        ykt_full_s = [entry[0] for entry in ykt.entryList]
        ykt_full_e = [entry[1] for entry in ykt.entryList]

        for s_point, e_point, label in a.entryList:
            ter = 0
            if s_point in cster_list:
                ter = 1
            if s_point in rus_full_s and e_point in rus_full_e:
                language = 'R'
            elif s_point in ykt_full_s and e_point in ykt_full_e:
                language = 'Y'
            elif s_point in rus_full_s and e_point in ykt_full_e:
                language = 'CS'
            elif s_point in ykt_full_s and e_point in rus_full_e:
                language = 'CS'
            else:
                language = 'N'
            c.execute('INSERT INTO replics VALUES (?, ?, ?, ?, ?)', (s_point, label, i, language, ter))
  
            
        conn.commit()
       
    c.execute('ATTACH DATABASE "new8.db" as new')
    c.execute('INSERT INTO new SELECT * FROM replics ORDER BY start_point ASC')
    conn.commit()  

    conn.close()
    
replics_db()    

# SELECT * FROM replics ORDER BY start_point ASC

In [4]:
# new db: a table
# WRONGLY UPDATED
def turns():
    conn = sqlite3.connect('new8.db')
    c = conn.cursor()   
    c.execute('SELECT * FROM new')
    a = c.fetchall()
    conn.close()
    n = len(a)
    conn = sqlite3.connect('turns5.db')
    c = conn.cursor() 
    c.execute('CREATE TABLE IF NOT EXISTS turns(start_pointL float, interL integer, speakerL integer, replicL text, languageL text, CSL integer, CSR integer, languageR text, replicR text, speakerR integer, interR integer, start_pointR float)')
    for i in range (0, n - 1):
        if a[i][2] != a[i + 1][2]:
            c.execute('INSERT INTO turns VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)', (a[i][0], a[i][4], a[i][2], a[i][1], a[i][3], a[i][4], a[i + 1][4], a[i + 1][3], a[i + 1][1], a[i + 1][2], a[i + 1][4], a[i + 1][0]))
        conn.commit()
    conn.close()
    
    
    
turns()